In [1]:
# 1. Import Essential Libraries
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns  # Optional: For better visualizations

# 2. Machine Learning Imports
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans, DBSCAN  # Clustering Algorithms
from sklearn.metrics import silhouette_score, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler  # Data normalization for clustering

# 3. Ensure Directories for Saving Models Exist
trial_pp_dir = os.path.join(os.getcwd(), "trial_pp")
trial_itt_dir = os.path.join(os.getcwd(), "trial_itt")
os.makedirs(trial_pp_dir, exist_ok=True)
os.makedirs(trial_itt_dir, exist_ok=True)

# 4. Define Trial Dictionaries (If Used Later)
trial_pp = {"estimand": "PP"}  
trial_itt = {"estimand": "ITT"}  


### 1. Data Preparation

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Load Dataset
data_censored = pd.read_csv("data_censored.csv")

# Define Features for Clustering
cluster_features = data_censored[["x1", "x2", "x3", "age"]]

# Standardize Features for Clustering (Important for Distance-Based Methods)
scaler = StandardScaler()
cluster_features_scaled = scaler.fit_transform(cluster_features)

# Apply K-Means Clustering
num_clusters = 3  # You can adjust this based on Elbow Method or other techniques
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
data_censored["cluster"] = kmeans.fit_predict(cluster_features_scaled)

# Compute Silhouette Score for Clustering Quality
silhouette_avg = silhouette_score(cluster_features_scaled, data_censored["cluster"])
data_censored["silhouette_score"] = silhouette_avg  # Storing overall score (not per row)

# Assign Dataset to Trial Objects
trial_pp["data"] = data_censored
trial_itt["data"] = data_censored

# Save Processed Data for Further Analysis
data_censored.to_csv("data_censored_clustered.csv", index=False)

# Print Summary
print(f"Clustering Completed. Silhouette Score: {silhouette_avg:.4f}")
print(data_censored.head())


We have made several improvements to the data preparation code to enhance its efficiency and accuracy. First, we standardized the selected features (`x1`, `x2`, `x3`, and `age`) using `StandardScaler` to ensure that K-Means clustering operates effectively, as it is sensitive to varying feature scales. Next, we corrected the calculation of the silhouette score by computing a single overall score for clustering quality, rather than assigning an individual score to each row. Additionally, we saved the processed dataset as `data_censored_clustered.csv`, allowing for further analysis and reproducibility. Finally, we included a print summary to provide immediate feedback on the clustering process and the computed silhouette score.

## 2. Weight Models and Calculation (With Clustering)  

In [ ]:
import statsmodels.api as sm

# Define Features and Target for Switch Weight Model
X_pp = data_censored[["age", "x1", "x3", "cluster"]]
y_pp = data_censored["treatment"]

# Fit Logistic Regression Model for Switch Weights
logit_model_pp = sm.Logit(y_pp, sm.add_constant(X_pp)).fit()
trial_pp["switch_weights"] = logit_model_pp.predict(sm.add_constant(X_pp))

# Define Features and Target for Censoring Weight Model
X_censor = data_censored[["x2", "x1", "cluster"]]
y_censor = 1 - data_censored["censored"]

# Fit Logistic Regression Model for Censoring Weights
logit_model_censor = sm.Logit(y_censor, sm.add_constant(X_censor)).fit()
trial_pp["censor_weights"] = logit_model_censor.predict(sm.add_constant(X_censor))

# Compute Final Weights
trial_pp["weights"] = trial_pp["switch_weights"] * trial_pp["censor_weights"]

# Print Model Summaries for Reference
print("\nSwitch Weight Model Summary:")
print(logit_model_pp.summary())

print("\nCensoring Weight Model Summary:")
print(logit_model_censor.summary())


We have improved the weight model and calculation code by enhancing clarity, structure, and readability. First, we explicitly defined the features and target variables for both the switch weight and censoring weight models, ensuring a clear distinction between the two. We maintained proper use of logistic regression by adding a constant term (`sm.add_constant()`) to account for the intercept in the models. Additionally, we included print statements to display the model summaries, allowing us to assess model performance and interpret the estimated coefficients. Finally, we ensured that the final weights were correctly computed by multiplying the predicted switch and censoring weights.

## 3. Specify Outcome Model  

In [ ]:
import statsmodels.api as sm

# Define Features and Target for Outcome Model
X_outcome = sm.add_constant(data_censored[["x2", "cluster"]])
y_outcome = data_censored["outcome"]

# Fit Logistic Regression Model for Outcome
logit_outcome_model = sm.Logit(y_outcome, X_outcome).fit()
trial_pp["outcome_model"] = logit_outcome_model.predict(X_outcome)

# Compute Pseudo R² (McFadden's R²)
pseudo_r2_original = 1 - (logit_outcome_model.llf / logit_outcome_model.llnull)

# Print Model Summary and Pseudo R²
print("\nOutcome Model Summary:")
print(logit_outcome_model.summary())
print(f"\nPseudo R² (McFadden's R²): {pseudo_r2_original:.4f}")


## 4. Expand, Sample, and Fit MSM  

In [5]:
import numpy as np
import statsmodels.api as sm

# Expand the dataset via bootstrapping
trial_pp["expanded_data"] = data_censored.sample(frac=1, replace=True, random_state=42).reset_index(drop=True)

# Sample a subset (50%) of the expanded dataset
trial_pp["sampled_data"] = trial_pp["expanded_data"].sample(frac=0.5, random_state=1234)

# Apply weight trimming to avoid extreme values
weights = trial_pp["weights"].astype(float)
trimmed_weight_threshold = np.quantile(weights, 0.99)
weights = np.minimum(weights, trimmed_weight_threshold)

# Match trimmed weights to sampled data
sampled_indices = trial_pp["sampled_data"].index
weights_sampled = weights.loc[sampled_indices].values  # Ensure it's an array for GLM

# Define MSM model using Generalized Linear Model (GLM)
X_msm = sm.add_constant(trial_pp["sampled_data"][["x2"]])  # Removing 'cluster' for baseline comparison
y_msm = trial_pp["sampled_data"]["outcome"]

# Fit MSM model with binomial family and weighted observations
msm_model = sm.GLM(y_msm, X_msm, family=sm.families.Binomial(), freq_weights=weights_sampled).fit()
trial_pp["msm_model"] = msm_model

# Compute Pseudo R² for MSM Model
pseudo_r2_clustered = 1 - (msm_model.llf / msm_model.llnull)

# Print Model Summary and Pseudo R²
print("\nMSM Model Summary:")
print(msm_model.summary())
print(f"\nPseudo R² (McFadden's R²) for MSM Model: {pseudo_r2_clustered:.4f}")


## 5. Alternative Clustering & Survival Prediction  

In [6]:
import numpy as np
import statsmodels.api as sm
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Apply DBSCAN Clustering
dbscan = DBSCAN(eps=1.5, min_samples=5)
data_censored["dbscan_cluster"] = dbscan.fit_predict(cluster_features)

# Compute Silhouette Score (only if DBSCAN has more than 1 cluster)
if len(set(data_censored["dbscan_cluster"])) > 1:
    dbscan_silhouette = silhouette_score(cluster_features, data_censored["dbscan_cluster"], metric="euclidean")
    print(f"DBSCAN Silhouette Score: {dbscan_silhouette:.4f}")
else:
    dbscan_silhouette = None
    print("DBSCAN resulted in only one cluster or noise points, Silhouette Score not applicable.")

# Predict survival probabilities using MSM Model
predict_times = np.arange(0, 11)
X_survival = sm.add_constant(trial_pp["sampled_data"]["x2"])
survival_predictions = msm_model.predict(X_survival)

# Display first few survival predictions
print("\nSurvival Predictions (First 10):")
print(survival_predictions[:10])


## 6. Model Comparison & Performance Evaluation  

In [7]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Compute AIC/BIC for Logit Outcome Model
original_aic = logit_outcome_model.aic
n = len(y_outcome)  # Number of observations
k = logit_outcome_model.df_model + 1  # Number of parameters (including intercept)
log_likelihood = logit_outcome_model.llf
original_bic = k * np.log(n) - 2 * log_likelihood

# Compute AIC/BIC for MSM Model
clustered_aic = msm_model.aic
n_msm = len(y_msm)
k_msm = msm_model.df_model + 1
log_likelihood_msm = msm_model.llf
clustered_bic = k_msm * np.log(n_msm) - 2 * log_likelihood_msm

# Compute AUC for both models
auc_original = roc_auc_score(y_outcome, logit_outcome_model.predict(X_outcome))
auc_clustered = roc_auc_score(y_msm, msm_model.predict(X_msm))

# Cross-Validation Setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Train Logistic Regression Models for CV Evaluation
logit_outcome_model_sklearn = LogisticRegression(max_iter=1000, solver="liblinear").fit(X_outcome, y_outcome)
msm_model_sklearn = LogisticRegression(max_iter=1000, solver="liblinear").fit(X_msm, y_msm)

# Compute Cross-Validated AUC Scores
cross_val_original = np.mean(cross_val_score(logit_outcome_model_sklearn, X_outcome, y_outcome, cv=cv, scoring="roc_auc"))
cross_val_clustered = np.mean(cross_val_score(msm_model_sklearn, X_msm, y_msm, cv=cv, scoring="roc_auc"))

# Print Results for Easy Comparison
print("\n🔹 Model Comparison Metrics 🔹")
print(f"Logit Model AIC: {original_aic:.4f} | BIC: {original_bic:.4f}")
print(f"MSM Model AIC: {clustered_aic:.4f} | BIC: {clustered_bic:.4f}")
print(f"AUC (Logit Model): {auc_original:.4f} | AUC (MSM Model): {auc_clustered:.4f}")
print(f"Cross-Validated AUC (Logit Model): {cross_val_original:.4f}")
print(f"Cross-Validated AUC (MSM Model): {cross_val_clustered:.4f}")
